<a href="https://colab.research.google.com/github/Leonardo624/Projeto-Alura---Agiliza-SUS/blob/main/Agiliza_SUS_021_FINAL_Copia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-

# Instalação de dependências
!pip install -q google-generativeai
!pip install -q timezonefinder geopy pytz

import google.generativeai as genai
from google.colab import userdata
from IPython.display import Markdown, display
from datetime import datetime
from timezonefinder import TimezoneFinder
from geopy.geocoders import Nominatim
import pytz

# Configuração da API Gemini
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
modelo = genai.GenerativeModel("gemini-2.0-flash")

def chamar_modelo(prompt):
    try:
        resposta = modelo.generate_content(prompt)
        if not resposta or not hasattr(resposta, 'text') or not resposta.text.strip():
            return None
        return resposta.text.strip()
    except Exception as e:
        print(f"[ERRO Gemini] {e}")
        return None

def agente_classificador_motivo(texto):
    texto = texto.lower()
    if any(p in texto for p in ["vacina", "vacinação", "dose", "imunização"]): return "vacina"
    if any(p in texto for p in ["exame", "sangue", "imagem", "raio-x", "rotina", "checkup", "mamografia", "ultrassom"]): return "exame"
    if any(p in texto for p in ["cardiologista", "endocrino", "especialista", "encaminhamento", "ortopedista"]): return "especialidade"
    if any(p in texto for p in ["dor", "febre", "tontura", "infarto", "queda", "desmaio"]): return "sintoma"
    return "avaliar"

def get_gravidade_emoji(gravidade):
    grav = gravidade.lower()
    if "grave" in grav: return "🔴 Grave"
    if "moderado" in grav: return "🟡 Moderado"
    return "🟢 Leve"

def micro_agente_gravidade(sintomas):
    prompt = f"Classifique os sintomas abaixo como Leve, Moderado ou Grave.\nSintomas: {sintomas}\nApenas a palavra."
    resultado = chamar_modelo(prompt)
    if resultado and resultado.strip().capitalize() in ["Leve", "Moderado", "Grave"]:
        return get_gravidade_emoji(resultado.strip())
    return "🟢 Leve"

# MICRO-AGENTES: Cada um orienta conforme o seu nível
def micro_agente_primaria(sintomas, idade, obs):
    prompt = f"""
Você é um agente SUS para Atenção Primária. Sintomas: {sintomas} | Idade: {idade} | Obs: {obs}
Oriente brevemente o usuário sobre quando e por que buscar a UBS. Seja direto e sem rodeios.
"""
    resposta = chamar_modelo(prompt)
    return resposta if resposta else "Procure a UBS (Atenção Primária) nos horários de funcionamento, levando documentos e exames prévios."

def micro_agente_secundaria(sintomas, idade, obs):
    prompt = f"""
Você é um agente SUS para Atenção Secundária. Sintomas: {sintomas} | Idade: {idade} | Obs: {obs}
Explique rapidamente quando procurar encaminhamento para especialista, exames complementares ou serviços secundários do SUS.
"""
    resposta = chamar_modelo(prompt)
    return resposta if resposta else "Procure a UBS para avaliação e, se necessário, solicite encaminhamento para o serviço especializado (Atenção Secundária)."

def micro_agente_terciaria(sintomas, idade, obs):
    prompt = f"""
Você é um agente SUS para Atenção Terciária. Sintomas: {sintomas} | Idade: {idade} | Obs: {obs}
Explique, de forma objetiva, quando procurar UPA ou Hospital. Seja prático e evite enrolação.
"""
    resposta = chamar_modelo(prompt)
    return resposta if resposta else "Procure a UPA ou Hospital imediatamente em caso de sintomas graves, risco de vida ou emergência."

# ORQUESTRADOR: Recebe as três respostas e entrega a orientação consolidada
def agente_orquestrador_final(nivel, resposta_primaria, resposta_secundaria, resposta_terciaria):
    bloco = "\n---\n"
    if "Primária" in nivel:
        return f"### 🏥 Orientação Consolidada\n{resposta_primaria}{bloco}Para outras necessidades, a UBS pode encaminhar ao especialista (Secundária) ou UPA/Hospital (Terciária) se houver risco."
    elif "Secundária" in nivel:
        return f"### 🏥 Orientação Consolidada\n{resposta_secundaria}{bloco}Caso evolua com sintomas graves, dirija-se à UPA ou Hospital (Terciária) imediatamente."
    elif "Terciária" in nivel:
        return f"### 🏥 Orientação Consolidada\n{resposta_terciaria}{bloco}Em casos menos graves ou de rotina, busque a UBS (Primária) para avaliação inicial."
    else:
        return f"### 🏥 Orientação Consolidada\nNão foi possível determinar o melhor nível. Procure a UBS para avaliação inicial."

def obter_fuso_pelo_cep(cep):
    if not cep: return pytz.timezone("America/Sao_Paulo")
    try:
        geo = Nominatim(user_agent="app_sus")
        local = geo.geocode(f"{cep}, Brasil")
        if not local: return pytz.timezone("America/Sao_Paulo")
        fuso = TimezoneFinder().timezone_at(lat=local.latitude, lng=local.longitude)
        return pytz.timezone(fuso) if fuso else pytz.timezone("America/Sao_Paulo")
    except: return pytz.timezone("America/Sao_Paulo")

def obter_hora_local(cep):
    return datetime.now(obter_fuso_pelo_cep(cep)).hour

def gerar_status_e_link(gravidade, tipo, cep, hora, dia):
    if "Grave" in gravidade or "Moderado" in gravidade:
        return "✅ UPA e hospitais funcionam 24h.", f"[UPA próxima (CEP {cep})](https://www.google.com/maps/search/UPA+perto+de+{cep}+Brasil)"
    if dia.lower() in ["saturday", "sunday"] or not (7 <= hora < 17):
        return "🚫 UBS fechada. Aguarde dia útil.", f"[UBS próxima (CEP {cep})](https://www.google.com/maps/search/UBS+perto+de+{cep}+Brasil)"
    return "✅ UBS aberta neste horário.", f"[UBS próxima (CEP {cep})](https://www.google.com/maps/search/UBS+perto+de+{cep}+Brasil)"

# ==============================
# ENTRADA DE DADOS DO USUÁRIO AO FINAL

print("👋 Bem-vindo ao Agiliza SUS!")
print("Preencha as informações abaixo para receber orientação do SUS.\n")

sintomas = input("🩺 Sintomas ou serviço desejado: ").strip()
cep = input("📍 CEP (opcional): ").strip()
idade = input("🎂 Idade (opcional): ").strip()
obs = input("📌 Condições clínicas (ex: gestante, hipertenso): ").strip()

motivo = agente_classificador_motivo(sintomas)
gravidade = micro_agente_gravidade(sintomas)
# CORRIGIDO: sem acento
def micro_agente_nivel_atencao(sintomas, observacoes, gravidade):
    prompt = f"Diga apenas o nível SUS adequado: Atenção Primária, Secundária ou Terciária.\nSintomas: {sintomas}\nObs: {observacoes}\nGravidade: {gravidade}"
    resultado = chamar_modelo(prompt)
    if resultado and any(n in resultado for n in ["Primária", "Secundária", "Terciária"]):
        return resultado.strip()
    return "Atenção Primária (UBS)"
nivel = micro_agente_nivel_atencao(sintomas, obs, gravidade)
hora = obter_hora_local(cep)
dia = datetime.now().strftime("%A")
status, link = gerar_status_e_link(gravidade, motivo, cep, hora, dia)

resposta_primaria = micro_agente_primaria(sintomas, idade, obs)
resposta_secundaria = micro_agente_secundaria(sintomas, idade, obs)
resposta_terciaria = micro_agente_terciaria(sintomas, idade, obs)

orientacao_final = agente_orquestrador_final(nivel, resposta_primaria, resposta_secundaria, resposta_terciaria)

resposta = f"""
### 🚦 Quadro de Gravidade
{gravidade}

### 🏥 Nível de Atenção Indicado
{nivel}

### 🕒 Status da Unidade
{status}

### 📌 Localização
{link}<br>
⚠️ O Google Maps pode mostrar locais privados. Para atendimento SUS, escolha apenas unidades públicas com identificação oficial do SUS.

{orientacao_final}

---

Orientação validada por agente digital de saúde pública.
"""

display(Markdown(resposta))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.8 MB/s eta 0:00:00


NotebookAccessError: Notebook does not have access to secret GOOGLE_API_KEY